<a href="https://colab.research.google.com/github/SurajQWERTY/d3_projects_files/blob/main/ResNet34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import keras and other libraries
import tensorflow as tf
import keras
from keras.datasets import cifar10
import numpy as np

In [3]:
# Implement a ResNet-34 CNN Using Keras
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides, padding='same', use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1, padding='same', use_bias=False),
            keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides, padding='same', use_bias=False),
                keras.layers.BatchNormalization()
            ]
    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[32, 32, 3], padding='same', use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same'))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 64)        9408      
                                                                 
 batch_normalization (BatchN  (None, 16, 16, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 16, 16, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 64)         0         
 )                                                               
                                                                 
 residual_unit (ResidualUnit  (None, 8, 8, 64)         74240     
 )                                                               
                                                        

In [6]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [9]:
# Compile the model
model.compile(optimizer= 'adam',loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

# Fit the trainining data in the Resnet model
model.fit(x_train, y_train, batch_size= 64, epochs= 50, validation_data=(x_test, y_test), shuffle=True)

Epoch 1/50
782/782 [==============================] - 3366s 4s/step - loss: 1.5382 - accuracy: 0.4574 - val_loss: 1.7016 - val_accuracy: 0.4224
Epoch 2/50
782/782 [==============================] - 3269s 4s/step - loss: 1.1288 - accuracy: 0.6032 - val_loss: 1.4608 - val_accuracy: 0.4976
Epoch 3/50
782/782 [==============================] - 3249s 4s/step - loss: 0.9838 - accuracy: 0.6569 - val_loss: 1.2421 - val_accuracy: 0.5771
Epoch 4/50
782/782 [==============================] - 3263s 4s/step - loss: 0.8188 - accuracy: 0.7147 - val_loss: 2.1483 - val_accuracy: 0.3898
Epoch 5/50
782/782 [==============================] - 3261s 4s/step - loss: 0.7568 - accuracy: 0.7394 - val_loss: 0.9116 - val_accuracy: 0.6876
Epoch 6/50
782/782 [==============================] - 3272s 4s/step - loss: 0.6790 - accuracy: 0.7667 - val_loss: 0.9510 - val_accuracy: 0.6829
Epoch 7/50
782/782 [==============================] - 3237s 4s/step - loss: 0.7040 - accuracy: 0.7601 - val_loss: 0.9388 - val_accuracy:

KeyboardInterrupt: ignored